In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!apt-get install -y coinor-cbc

In [ ]:
import pandas as pd

df_clients = pd.read_csv('./content/base/clients.csv')
df_depots = pd.read_csv('./content/base/depots.csv')
df_vehicles = pd.read_csv('./content/base/vehicles.csv')

In [ ]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [ ]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C


print("model.D =", list(model.D.data()))
print("model.C =", list(model.C.data()))
print("model.V =", list(model.V.data()))
print("model.N =", list(model.N.data()))
print((lugares))

model.D = ['CD1']
model.C = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
model.V = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8']
model.N = ['CD1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
[(-74.153536, 4.743359), (-74.09893796560621, 4.59795431125545), (-74.07557103763986, 4.687820646838871), (-74.10708524062704, 4.70949446000624), (-74.09727965657427, 4.605029068682624), (-74.16464148202755, 4.648463876533332), (-74.12083799988112, 4.662137416953968), (-74.02213076607309, 4.697499030379109), (-74.17207549744595, 4.649416884236942), (-74.15615257246444, 4.606310650273935), (-74.09041145358674, 4.557379705282216), (-74.17802255204528, 4.591594072172954), (-74.1015410917749, 4.7564172406324055), (-74.09690889182339, 4.646217006050524), (-74.1219200708342, 

In [ ]:
import openrouteservice
from openrouteservice import convert
import json

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'

client = openrouteservice.Client(key=API_KEY)

matrix = client.distance_matrix(
    locations=lugares,
    profile='driving-car',
    metrics=['distance'],
    resolve_locations=True,
    units='km',
)

In [ ]:
precio_km = 3032.1
cost = {}
distance = {}

dist_matrix = matrix['distances']
print(dist_matrix)
print(len(dist_matrix))

for i,i1 in zip(range(len(lugares)),  list(model.N.data())):
        for j,j1 in zip(range(len(lugares)),  list(model.N.data())):
            if i != j:
                distancia_km = dist_matrix[i][j]
                precio = round(distancia_km * precio_km, 3)
            else:
                distancia_km = 9999999
                precio =9999999
            distance[(i1, j1)] = distancia_km
            cost[(i1, j1)] = precio
print(distance)
print(cost)

[[0.0, 27.19, 16.99, 12.37, 26.7, 22.64, 19.65, 25.02, 23.89, 30.67, 33.86, 33.49, 15.87, 21.66, 10.36, 26.79, 33.91, 26.61, 19.87, 20.93, 13.14, 27.84, 24.41, 11.34, 30.87], [30.96, 0.0, 14.29, 19.44, 1.13, 12.5, 11.32, 17.81, 13.75, 7.51, 7.39, 10.33, 25.81, 8.3, 20.31, 1.29, 7.44, 4.49, 9.84, 14.97, 23.09, 21.68, 5.38, 21.29, 4.89], [18.34, 12.69, 0.0, 6.83, 12.2, 14.3, 7.43, 9.22, 15.55, 15.46, 19.36, 18.27, 13.19, 6.44, 7.69, 12.29, 19.41, 11.39, 8.78, 5.6, 10.47, 12.8, 9.91, 8.67, 15.81], [13.76, 15.97, 5.77, 0.0, 15.48, 11.42, 8.43, 13.8, 12.67, 19.45, 22.64, 22.27, 8.62, 10.44, 3.11, 15.57, 22.69, 15.39, 8.65, 9.71, 5.89, 16.61, 13.19, 4.09, 19.64], [29.95, 1.05, 13.28, 18.43, 0.0, 11.49, 10.31, 16.81, 12.74, 7.9, 8.32, 10.71, 24.8, 7.29, 19.3, 1.25, 8.37, 4.17, 8.83, 13.97, 22.08, 20.68, 4.38, 20.27, 5.27], [21.07, 13.4, 16.25, 17.53, 12.91, 0.0, 6.96, 22.65, 1.54, 7.17, 20.06, 9.56, 22.37, 13.33, 18.39, 13.0, 16.51, 10.38, 6.73, 18.56, 21.17, 25.46, 12.83, 19.37, 14.8], [23.0

In [ ]:
depot_capacity = {}
for depot in df_depots['DepotID']:
    depot_capacity[f'CD{depot}'] = 9999999

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)

{'CD1': 9999999}
{'C1': 13, 'C2': 15, 'C3': 12, 'C4': 15, 'C5': 20, 'C6': 17, 'C7': 17, 'C8': 20, 'C9': 20, 'C10': 15, 'C11': 17, 'C12': 12, 'C13': 21, 'C14': 15, 'C15': 17, 'C16': 10, 'C17': 25, 'C18': 12, 'C19': 11, 'C20': 15, 'C21': 14, 'C22': 18, 'C23': 15, 'C24': 11}
{'V1': 130, 'V2': 140, 'V3': 120, 'V4': 100, 'V5': 70, 'V6': 55, 'V7': 110, 'V8': 114}
{'V1': 170, 'V2': 200, 'V3': 180, 'V4': 90, 'V5': 100, 'V6': 170, 'V7': 150, 'V8': 140}


In [ ]:
# Parámetros
model.cost = Param(model.N, model.N, initialize=lambda model, i, j: cost[(i, j)], within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda model, i, j: distance[(i, j)], within=NonNegativeReals)

# Variables de decisión y auxiliares

model.x = Var(model.V, model.N, model.N, domain=Binary)
model.y = Var(model.V, model.D, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# funcion objetivo

def obj_rule(model):
    return sum(model.cost[i,j] * model.x[v,i,j] for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

def assign_depot_rule(model, v):
    return sum(model.y[v,d] for d in model.D) == 1

model.assign_depot = Constraint(model.V, rule=assign_depot_rule)

def start_route_rule(model, v, d):
    return sum(model.x[v, d, j] for j in model.N if j != d) == model.y[v,d]

model.start_route = Constraint(model.V, model.D, rule=start_route_rule)

"""
def return_route_rule(model, v, d):
    return sum(model.x[v, i, d] for i in model.N if i != d) == model.y[v,d]

model.return_route = Constraint(model.V, model.D, rule=return_route_rule)
"""

def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    else:
        return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

def no_depot_to_depot_rule(model, v, i, j):
    if i in model.D and j in model.D and i != j:
        return model.x[v,i,j] == 0
    else:
        return Constraint.Skip

model.no_depot_travel = Constraint(model.V, model.N, model.N, rule=no_depot_to_depot_rule)

def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    else:
        return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

def depot_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= sum(model.y[v,d] * depot_capacity[d] for d in model.D)

model.depot_capacity = Constraint(model.V, rule=depot_capacity_rule)

def vehicle_range_rule(model, v):
    return sum(model.distance[i,j] * model.x[v,i,j] for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)

def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v,i] - model.u[v,j] + vehicle_capacity[v] * model.x[v,i,j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v,i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v,i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

# Resolver
from amplpy import modules
#solver = SolverFactory('cbc')
#solver.options["seconds"] = 300  # máximo 5 minutos
solver_name = "gurobi"
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
solver.options['TimeLimit'] = 300  # 5 minutos de tiempo máximo
solver.options['MIPGap'] = 0.01  # Establecer un límite en el gap de optimalidad
results = solver.solve(model)

model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Gurobi 12.0.1\x3a time limit, feasible solution;
      objective 1074758.166; 529234 simplex iterations; 22309 branching nodes;
      absmipgap=90387, relmipgap=0.0840999


In [ ]:
from pyomo.environ import value

def print_routes(model):
    print(f"\n==== Costo óptimo total del plan de rutas: $ {value(model.obj):.3f}")
    for v in model.V:
        print(f"\n>>> Ruta del vehículo {v}:")

        # 1. Encontrar el depósito asignado al vehículo y su capacidad
        depot = None
        for d in model.D:
            if value(model.y[v, d]) > 0.5:
                depot = d
                break

        if depot is None:
            print("Vehículo no asignado a ningún depósito.")
            continue

        print(f" - Depósito de salida: {depot} (Capacidad depósito: {depot_capacity[depot]})")
        print(f" - Capacidad del vehículo: {vehicle_capacity[v]} unidades")
        print(f" - Rango operativo del vehículo: {vehicle_range[v]} km")

        # 2. Reconstruir la ruta
        current_node = depot
        route = [current_node]
        visited = set()
        deliveries = {}

        while True:
            next_node = None
            for j in model.N:
                if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                    next_node = j
                    break

            if next_node is None or next_node in visited:
                break

            if next_node in model.C:
                deliveries[(current_node, next_node)] = demand[next_node]

            route.append(next_node)
            visited.add(next_node)
            current_node = next_node

        # 3. Calcular carga total y distancia total
        total_load = sum(deliveries.values())
        total_distance = sum(
            value(model.distance[route[i], route[i+1]])
            for i in range(len(route)-1)
        )

        # 4. Imprimir resultados
        if len(route) > 1:
            print(f" - Ruta: {' → '.join(route)}")
            print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]} unidades")
            print(f" - Distancia total recorrida: {total_distance:.3f} km")
            porcentaje = total_distance / vehicle_range[v] * 100
            print(f" - Uso del rango operativo: {porcentaje:.1f}%")
            print(f' - Costo del transporte: $ {total_distance*precio_km:.2f}')

            print(" - Entregas entre nodos:")
            for (i, j), qty in deliveries.items():
                print(f"    De {i} → {j}: {qty} unidades")
        else:
            print("Vehículo no utilizado.")
print_routes(model)


==== Costo óptimo total del plan de rutas: $ 1074758.166

>>> Ruta del vehículo V1:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 130 unidades
 - Rango operativo del vehículo: 170 km
 - Ruta: CD1 → C12 → CD1
 - Carga total transportada: 12/130 unidades
 - Distancia total recorrida: 31.890 km
 - Uso del rango operativo: 18.8%
 - Costo del transporte: $ 96693.67
 - Entregas entre nodos:
    De CD1 → C12: 12 unidades

>>> Ruta del vehículo V2:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 140 unidades
 - Rango operativo del vehículo: 200 km
 - Ruta: CD1 → C6 → C17 → C24 → C16 → C10 → C1 → C15 → C4 → C18 → CD1
 - Carga total transportada: 135/140 unidades
 - Distancia total recorrida: 77.140 km
 - Uso del rango operativo: 38.6%
 - Costo del transporte: $ 233896.19
 - Entregas entre nodos:
    De CD1 → C6: 17 unidades
    De C6 → C17: 25 unidades
    De C17 → C24: 11 unidades
    De C24 → C16: 10 unidades
    De C1

In [ ]:
import folium
import openrouteservice
from openrouteservice import convert
from pyomo.environ import value
import itertools

# Paleta de colores (puedes expandirla si tienes más vehículos)
colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'black', 'pink', 'gray', 'cyan']
vehiculo_a_color = dict(zip(model.V, itertools.cycle(colores)))


client = openrouteservice.Client(key=API_KEY)

m = folium.Map(location=[4.65, -74.1], zoom_start=11)

coords= lugares
labels= list(model.N.data())


In [ ]:
for label, coord in zip(labels, coords):
    folium.Marker(location=(coord[1], coord[0]), popup=label).add_to(m)

# Diccionario de índice a coordenada
label_to_coord = dict(zip(labels, coords))

# Supón que tienes model.x[v, i, j] y model.V y model.N cargado
# Simulación: rutas como lista de tuplas (vehículo, origen, destino)
rutas_optimas = []
for v in model.V:
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                rutas_optimas.append((v, i, j))
print(rutas_optimas)


[('V1', 'CD1', 'C12'), ('V1', 'C12', 'CD1'), ('V2', 'CD1', 'C6'), ('V2', 'C1', 'C15'), ('V2', 'C4', 'C18'), ('V2', 'C6', 'C17'), ('V2', 'C10', 'C1'), ('V2', 'C15', 'C4'), ('V2', 'C16', 'C10'), ('V2', 'C17', 'C24'), ('V2', 'C18', 'CD1'), ('V2', 'C24', 'C16'), ('V3', 'CD1', 'C23'), ('V3', 'C23', 'CD1'), ('V4', 'CD1', 'C22'), ('V4', 'C5', 'CD1'), ('V4', 'C8', 'C5'), ('V4', 'C9', 'C8'), ('V4', 'C11', 'C9'), ('V4', 'C22', 'C11'), ('V5', 'CD1', 'C3'), ('V5', 'C3', 'CD1'), ('V6', 'CD1', 'C14'), ('V6', 'C14', 'CD1'), ('V7', 'CD1', 'C20'), ('V7', 'C20', 'CD1'), ('V8', 'CD1', 'C7'), ('V8', 'C2', 'C13'), ('V8', 'C7', 'C21'), ('V8', 'C13', 'CD1'), ('V8', 'C19', 'C2'), ('V8', 'C21', 'C19')]


In [ ]:
for (v, i, j) in rutas_optimas:
    coord_i = label_to_coord[i]
    coord_j = label_to_coord[j]
    color = vehiculo_a_color[v]
    try:
        route = client.directions(
            coordinates=[coord_i, coord_j],
            profile='driving-car',
            format='geojson'
        )
        folium.GeoJson(
            route,
            name=f"Ruta {v}: {i}->{j}",
            tooltip=f"{v}: {i} → {j}",
            style_function=lambda x, col=color: {'color': col, 'weight': 4, 'opacity': 0.8}
        ).add_to(m)
    except Exception as e:
        print(f"Error en ruta {i} → {j}: {e}")
m

In [1]:
import re

# Texto de entrada (sustituye "..." por el texto completo que copiaste del plan de rutas)
raw_text = """==== Costo óptimo total del plan de rutas: $ 1074758.166

>>> Ruta del vehículo V1:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 130 unidades
 - Rango operativo del vehículo: 170 km
 - Ruta: CD1 → C12 → CD1
 - Carga total transportada: 12/130 unidades
 - Distancia total recorrida: 31.890 km
 - Uso del rango operativo: 18.8%
 - Costo del transporte: $ 96693.67
 - Entregas entre nodos:
    De CD1 → C12: 12 unidades

>>> Ruta del vehículo V2:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 140 unidades
 - Rango operativo del vehículo: 200 km
 - Ruta: CD1 → C6 → C17 → C24 → C16 → C10 → C1 → C15 → C4 → C18 → CD1
 - Carga total transportada: 135/140 unidades
 - Distancia total recorrida: 77.140 km
 - Uso del rango operativo: 38.6%
 - Costo del transporte: $ 233896.19
 - Entregas entre nodos:
    De CD1 → C6: 17 unidades
    De C6 → C17: 25 unidades
    De C17 → C24: 11 unidades
    De C24 → C16: 10 unidades
    De C16 → C10: 15 unidades
    De C10 → C1: 13 unidades
    De C1 → C15: 17 unidades
    De C15 → C4: 15 unidades
    De C4 → C18: 12 unidades

>>> Ruta del vehículo V3:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 120 unidades
 - Rango operativo del vehículo: 180 km
 - Ruta: CD1 → C23 → CD1
 - Carga total transportada: 15/120 unidades
 - Distancia total recorrida: 23.580 km
 - Uso del rango operativo: 13.1%
 - Costo del transporte: $ 71496.92
 - Entregas entre nodos:
    De CD1 → C23: 15 unidades

>>> Ruta del vehículo V4:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 100 unidades
 - Rango operativo del vehículo: 90 km
 - Ruta: CD1 → C22 → C11 → C9 → C8 → C5 → CD1
 - Carga total transportada: 95/100 unidades
 - Distancia total recorrida: 73.130 km
 - Uso del rango operativo: 81.3%
 - Costo del transporte: $ 221737.47
 - Entregas entre nodos:
    De CD1 → C22: 18 unidades
    De C22 → C11: 17 unidades
    De C11 → C9: 20 unidades
    De C9 → C8: 20 unidades
    De C8 → C5: 20 unidades

>>> Ruta del vehículo V5:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 70 unidades
 - Rango operativo del vehículo: 100 km
 - Ruta: CD1 → C3 → CD1
 - Carga total transportada: 12/70 unidades
 - Distancia total recorrida: 26.130 km
 - Uso del rango operativo: 26.1%
 - Costo del transporte: $ 79228.77
 - Entregas entre nodos:
    De CD1 → C3: 12 unidades

>>> Ruta del vehículo V6:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 55 unidades
 - Rango operativo del vehículo: 170 km
 - Ruta: CD1 → C14 → CD1
 - Carga total transportada: 15/55 unidades
 - Distancia total recorrida: 21.570 km
 - Uso del rango operativo: 12.7%
 - Costo del transporte: $ 65402.40
 - Entregas entre nodos:
    De CD1 → C14: 15 unidades

>>> Ruta del vehículo V7:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 110 unidades
 - Rango operativo del vehículo: 150 km
 - Ruta: CD1 → C20 → CD1
 - Carga total transportada: 15/110 unidades
 - Distancia total recorrida: 26.220 km
 - Uso del rango operativo: 17.5%
 - Costo del transporte: $ 79501.66
 - Entregas entre nodos:
    De CD1 → C20: 15 unidades

>>> Ruta del vehículo V8:
 - Depósito de salida: CD1 (Capacidad depósito: 9999999)
 - Capacidad del vehículo: 114 unidades
 - Rango operativo del vehículo: 140 km
 - Ruta: CD1 → C7 → C21 → C19 → C2 → C13 → CD1
 - Carga total transportada: 78/114 unidades
 - Distancia total recorrida: 74.800 km
 - Uso del rango operativo: 53.4%
 - Costo del transporte: $ 226801.08
 - Entregas entre nodos:
    De CD1 → C7: 17 unidades
    De C7 → C21: 14 unidades
    De C21 → C19: 11 unidades
    De C19 → C2: 15 unidades
    De C2 → C13: 21 unidades"""  # ← Pega aquí el texto del plan de rutas

vehicles_data = []

vehicle_blocks = raw_text.strip().split(">>> Ruta del vehículo ")
for block in vehicle_blocks[1:]:
    lines = block.strip().split('\n')
    vehicle_number = re.search(r"V(\d+):", lines[0]).group(1)
    vehicle_id = f"VEH{int(vehicle_number):03d}"

    cap = int(re.search(r"Capacidad del vehículo: (\d+)", block).group(1))
    fuel = int(re.search(r"Rango operativo del vehículo: (\d+)", block).group(1))
    route = re.search(r"Ruta: (.+)", block).group(1).strip()
    route_ids = re.findall(r"C\d+|CD1", route)
    route_sequence = '-'.join(['CDA' if x == 'CD1' else f"C{x[1:].zfill(3)}" for x in route_ids])

    clients = [x for x in route_ids if x != 'CD1']
    client_count = len(clients)

    demands_raw = re.findall(r"De [^:]+: (\d+) unidades", block)
    demands = '-'.join(demands_raw)

    distance = re.search(r"Distancia total recorrida: ([\d.]+)", block).group(1)
    cost = re.search(r"Costo del transporte: \$ ([\d.]+)", block).group(1)

    vehicles_data.append([
        vehicle_id, cap, fuel, route_sequence, client_count, demands,
        cap, fuel, distance, 0, cost
    ])

# Generar el contenido del archivo
header = "VehicleId,LoadCap,FuelCap,RouteSequence,Municipalities,DemandSatisfied,InitialLoad,InitialFuel,Distance,Time,TotalCost"
lines = [header] + [','.join(map(str, row)) for row in vehicles_data]
output_text = '\n'.join(lines)

# Guardar en archivo
with open("respuesta.txt", "w", encoding="utf-8") as f:
    f.write(output_text)

# También imprimir en consola
print(output_text)


VehicleId,LoadCap,FuelCap,RouteSequence,Municipalities,DemandSatisfied,InitialLoad,InitialFuel,Distance,Time,TotalCost
VEH001,130,170,CDA-C012-CDA,1,12,130,170,31.890,0,96693.67
VEH002,140,200,CDA-C006-C017-C024-C016-C010-C001-C015-C004-C018-CDA,9,17-25-11-10-15-13-17-15-12,140,200,77.140,0,233896.19
VEH003,120,180,CDA-C023-CDA,1,15,120,180,23.580,0,71496.92
VEH004,100,90,CDA-C022-C011-C009-C008-C005-CDA,5,18-17-20-20-20,100,90,73.130,0,221737.47
VEH005,70,100,CDA-C003-CDA,1,12,70,100,26.130,0,79228.77
VEH006,55,170,CDA-C014-CDA,1,15,55,170,21.570,0,65402.40
VEH007,110,150,CDA-C020-CDA,1,15,110,150,26.220,0,79501.66
VEH008,114,140,CDA-C007-C021-C019-C002-C013-CDA,5,17-14-11-15-21,114,140,74.800,0,226801.08
